In [ ]:
# Modelo de recomendación basdo en usuarios

In [ ]:
!pip install scikit-surprise

  Using cached scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl


In [ ]:
!pip install joblib

In [ ]:
import pandas as pd
import numpy as np
import joblib
import pickle
from scipy import sparse
from surprise import Dataset, Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from joblib import parallel_backend

In [ ]:
qryUsrs = """
Select user_id
    ,gmap_id
    ,rating
from delorean_dataset.reviews_definitivos_Google
"""
dfUsers = pd.read_gbq(qryUsrs, dialect='standard')

In [ ]:
# Tomar una muestra aleatoria del 20% de los datos
dfUsers = dfUsers.sample(frac=0.2)

In [ ]:
# Total las valoraciones mayores a 3
res = (
  dfUsers
 .query('`rating` >= 3')
)

In [ ]:
# Convierte el dataframe a algo legible por surprise
# Las calificaciones fueron tomadas de los valores diferentes de la col, rating
reader = Reader(line_format='user item rating', rating_scale=(1, 5))
data = Dataset.load_from_df(res, reader)

In [ ]:
# Crea los set de entreno y prueba
set_entreno, set_prueba = train_test_split(data, test_size=.20)

In [ ]:
# Instancia el modelo
modeloUsr= SVD()

In [ ]:
# Entrena el modelo
modeloUsr.fit(set_entreno)

In [ ]:
# Hacemos una predicción sobre el set de prueba
predictions = modeloUsr.test(set_prueba)

In [ ]:
# Calcula el error absoluto medio
accuracy.mae(predictions)

MAE:  0.5696


0.5695724517017111

In [ ]:
# Muestra una predicción cualquiera del conjunto de prueba\n",
predictions[10]

Prediction(uid=1.182920086e+20, iid='0x80803550e8d2cbd9:0xbc89dc0b1315dae7', r_ui=5.0, est=4.548114093127432, details={'was_impossible': False})

In [ ]:
# Encontrar una mejor parametrización
from surprise.model_selection import cross_validate

rmse_test_means = [] # realizar validación cruzada y devolver los resultados de las métricas de error de los modelos
#factores = [1,2,4,8,16,32,64,128,256]
factores = [1,2,4,8,16,32] #  lista de valores para la cantidad de factores latentes del modelo

for factor in factores: 
    print ('==================================================')
    print('Evaluando con ',factor,' factores')
    modelo_evaluado = SVD(n_factors=factor) # se crea un modelo SVD con la cantidad de factores latentes
    resultado_cv = cross_validate(modelo_evaluado, data, measures=['RMSE', 'MAE'], cv = 5, verbose=True) # aplica la función cross_validate para este modelo con 5 particiones de validación cruzada
    rmse_test_means.append(np.mean(resultado_cv['test_rmse'])) # devuelve los resultados de las métricas de error RMSE y MAE para cada partición

Evaluando con  1  factores
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.6679  0.6707  0.6723  0.6699  0.6695  0.6701  0.0015  
MAE (testset)     0.5689  0.5708  0.5715  0.5698  0.5697  0.5701  0.0009  
Fit time          8.48    8.62    13.58   12.76   9.12    10.51   2.19    
Test time         0.94    0.91    1.38    0.92    0.91    1.01    0.18    
Evaluando con  2  factores
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.6716  0.6686  0.6715  0.6692  0.6708  0.6703  0.0012  
MAE (testset)     0.5713  0.5694  0.5710  0.5698  0.5707  0.5704  0.0007  
Fit time          9.01    13.40   13.15   9.32    9.21    10.82   2.01    
Test time         0.90    1.35    0.87    1.41    0.89    1.09    0.24    
Evaluando con  4  factores
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

       

In [ ]:
# Con los parámetros encontrados, busca el mejor modelo
parametros = {'n_factors': [5,50,100],'n_epochs': [5, 10,20], 'lr_all': [0.001, 0.002, 0.005],
              'reg_all': [0.002, 0.02, 0.2]} # diccionario de parámetros con diferentes valores para la cantidad de factores latentes, la cantidad de épocas, la tasa de aprendizaje y la regularización
gs = GridSearchCV(SVD, parametros, measures=['RMSE', 'MAE'], cv=3, n_jobs = -1) # buscar el mejor modelo basado en estos parámetros
gs.fit(data)

print('mejores puntajes : rmse ->',gs.best_score['rmse'],' mae ->',gs.best_score['mae'])
print('mejores parámetros : rmse ->',gs.best_params['rmse'],' mae ->',gs.best_params['mae'])

mejores puntajes : rmse -> 0.6708501240485192  mae -> 0.5724308090415433
mejores parámetros : rmse -> {'n_factors': 5, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.2}  mae -> {'n_factors': 5, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.002}


In [ ]:
# Modelo con los parámetros encontrado
modelo_mejor_evaluado = SVD(n_factors=5, n_epochs=5, lr_all=0.005, reg_all=0.002)

In [ ]:
# El mejor modelo encontrado
modelo_mejor_evaluado= gs.best_estimator['rmse']
modelo_mejor_evaluado.fit(set_entreno)
predictions = modelo_mejor_evaluado.test(set_prueba)

In [ ]:
# Guardar modelo
joblib.dump(modelo_mejor_evaluado, 'modelo.joblib')

['modelo.joblib']

In [ ]:
# Crea una instancia del cliente de Cloud Storage
client = storage.Client()

In [ ]:
# Carga el archivo de modelo a subir
bucket_name = "metadata_bucket1"
blob_name = "modelos/modelo.joblib"
local_file_path = "modelo.joblib"

In [ ]:

# Crea un objeto de Bucket
bucket = client.get_bucket(bucket_name)

In [ ]:
# Crea un objeto Blob y lo carga en el bucket
blob = bucket.blob(blob_name)
blob.upload_from_filename(local_file_path)

In [ ]:
--------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Crear un cliente de storage
client = storage.Client()

In [ ]:
# Carga el archivo de modelo a subir
bucket_name = "metadata_bucket1"
blob_name1 = "modelos/modelo.pickle"
local_file_path1 = "modelo.pickle"

In [ ]:
# Crea un objeto de Bucket
bucket = client.get_bucket(bucket_name)

In [ ]:
# Crea un objeto Blob y lo carga en el bucket
blob = bucket.blob(blob_name1)
blob.upload_from_filename(local_file_path1)